# Reading, processing, and writing large files

In this section, we will look at our second strategy for working with large files, namely reading a file in chunks, processing each chunk, then appending the results to the same file.

In [1]:
import pandas as pd
from dfply import *

## Hiding stack traceback

We hide the exception traceback for didactic reasons (code source: [see this post](https://stackoverflow.com/questions/46222753/how-do-i-suppress-tracebacks-in-jupyter)).  Don't run this cell if you want to see a full traceback.

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

## Example 2 - Adding some dateparts and writing out the result 

Now suppose that instead of aggregating and visualizing, our goal was to add some new columns to the data set and write the result to a csv.  Again, we will start by prototyping our code on the first chunk then transforming and writing all of the chunks. 

## Summary of the process

* Read and prototype on the first chunk
    * Outcome: helper functions for processing each chunk
* Reset the data frame iterator
* Process and write all chunks

#### Step 1 - Prototype on the first chunk

In [3]:
c_size = 10000
new_names = ['date', 'lat', 'lon', 'base']
date_cols = ['date']
df_iter = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-apr14.csv", 
                      header=0, names=new_names, 
                      parse_dates=date_cols,
                      chunksize=c_size,
                      sep=',',
                      engine='python')

In [4]:
from toolz import first
first_chunk = first(df_iter) 
first_chunk.dtypes

date    datetime64[ns]
lat            float64
lon            float64
base            object
dtype: object

#### Build an expression

In [5]:
(first_chunk 
 >> mutate(weekday = X.date.dt.weekday_name,
           weekofyear = X.date.dt.weekofyear,
           day = X.date.dt.day,
           hour = X.date.dt.hour) 
 >> head)

,date,lat,lon,base,weekday,weekofyear,day,hour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Tuesday,14,1,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Tuesday,14,1,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,Tuesday,14,1,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,Tuesday,14,1,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,Tuesday,14,1,0


#### Encapsulate the expression in a function and test on the first chunk

In [6]:
from functoolz import pipeable
add_dateparts = pipeable(lambda df: (df 
                                     >> mutate(weekday = X.date.dt.weekday_name,
                                               weekofyear = X.date.dt.weekofyear,
                                               dayofyear = X.date.dt.dayofyear,
                                               year = X.date.dt.year,
                                               month = X.date.dt.month_name(),
                                               day = X.date.dt.day,
                                               hour = X.date.dt.hour)))
first_chunk >> add_dateparts >> head

,date,lat,lon,base,weekday,weekofyear,dayofyear,year,month,day,hour
0,2014-04-01 00:11:00,40.7690,-73.9549,B02512,Tuesday,14,91,2014,April,1,0
1,2014-04-01 00:17:00,40.7267,-74.0345,B02512,Tuesday,14,91,2014,April,1,0
2,2014-04-01 00:21:00,40.7316,-73.9873,B02512,Tuesday,14,91,2014,April,1,0
3,2014-04-01 00:28:00,40.7588,-73.9776,B02512,Tuesday,14,91,2014,April,1,0
4,2014-04-01 00:33:00,40.7594,-73.9722,B02512,Tuesday,14,91,2014,April,1,0


#### Process and write the first chunk

For the first chunk, use

* `header=True`
* `mode='w'`
    * `'w'` == Write $\rightarrow$ creates a new file

In [7]:
out_file = "./data/uber-raw-data-apr14-with-datepart.csv"
add_dateparts(first_chunk).to_csv(out_file, header=True, mode='w', index=False)

#### Process and write the remaining chunks

For the remaining chunks, use

* `mode='a'`
    * `'a'` == append $\rightarrow$ adds lines to existing file
* `header=False`
    * No headers in the middle of the file

In [8]:
for i, chunk in enumerate(df_iter):
    print("writing chunk {0}".format(i+1))
    add_dateparts(chunk).to_csv(out_file, header=False, mode='a', index=False)

writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chunk 11
writing chunk 12
writing chunk 13
writing chunk 14
writing chunk 15
writing chunk 16
writing chunk 17
writing chunk 18
writing chunk 19
writing chunk 20
writing chunk 21
writing chunk 22
writing chunk 23
writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
writing chunk 40
writing chunk 41
writing chunk 42
writing chunk 43
writing chunk 44
writing chunk 45
writing chunk 46
writing chunk 47
writing chunk 48
writing chunk 49
writing chunk 50
writing chunk 51
writing chunk 52
writing chunk 53
writing chunk 54
writing chunk 55
writing chunk 56


In [10]:
!head -n 10 ./data/uber-raw-data-apr14-with-datepart.csv

date,lat,lon,base,weekday,weekofyear,dayofyear,year,month,day,hour
2014-04-01 00:11:00,40.769,-73.9549,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:17:00,40.7267,-74.0345,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:21:00,40.7316,-73.9873,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:28:00,40.7588,-73.9776,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:33:00,40.7594,-73.9722,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:33:00,40.7383,-74.0403,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:39:00,40.7223,-73.9887,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:45:00,40.762,-73.979,B02512,Tuesday,14,91,2014,April,1,0
2014-04-01 00:55:00,40.7524,-73.99600000000001,B02512,Tuesday,14,91,2014,April,1,0


## A note on out of memory errors

* Happen frequently when reading data in chunks

#### Example - MoMA Artwork

Even though this is not large (easily fits in memory on modern machines), we get memory errors when iterating through the chunks.

In [11]:
"""[chunk for chunk in pd.read_csv('./data/Artworks.csv', chunksize=500)]"""

"[chunk for chunk in pd.read_csv('./data/Artworks.csv', chunksize=500)]"

## Solution 1 - Specify the `sep` and let `engine='python'`

In [12]:
"""df_iter = pd.read_csv('./data/Artworks.csv', 
                      chunksize=500, # Pick a reasonable chunk size.  I had memory errors with a smaller size
                      sep=',', # To help the parser not run out of memory
                      dtype={'BeginDate':str}, # We are using string method, make sure they will work
                      engine='python') # The way I fixed parsing errors
[chunk for chunk in pd.read_csv('./data/Artworks.csv', chunksize=500)]"""

"df_iter = pd.read_csv('./data/Artworks.csv', \n                      chunksize=500, # Pick a reasonable chunk size.  I had memory errors with a smaller size\n                      sep=',', # To help the parser not run out of memory\n                      dtype={'BeginDate':str}, # We are using string method, make sure they will work\n                      engine='python') # The way I fixed parsing errors\n[chunk for chunk in pd.read_csv('./data/Artworks.csv', chunksize=500)]"

## Solution 2 - Use `csv.reader` or `csv.DictReader`

In [13]:
"""import pandas as pd
from csv import DictReader, Sniffer
from toolz import partition_all

with open('./data/Artworks.csv') as csvfile:
    dialect = Sniffer().sniff(csvfile.read(50))
    csvfile.seek(0)
    reader = DictReader(csvfile, dialect=dialect)
    columns = reader.fieldnames
    chunksize = 10000
    for i, chunk in enumerate(partition_all(chunksize, reader)):
        print('creating df {0}'.format(i))
        _ = pd.DataFrame().from_dict(chunk)"""

"import pandas as pd\nfrom csv import DictReader, Sniffer\nfrom toolz import partition_all\n\nwith open('./data/Artworks.csv') as csvfile:\n    dialect = Sniffer().sniff(csvfile.read(50))\n    csvfile.seek(0)\n    reader = DictReader(csvfile, dialect=dialect)\n    columns = reader.fieldnames\n    chunksize = 10000\n    for i, chunk in enumerate(partition_all(chunksize, reader)):\n        print('creating df {0}'.format(i))\n        _ = pd.DataFrame().from_dict(chunk)"

## What does `DictReader` do?

* Read a line at a time
* Return a row `dict` of `(col_name, value)` pairs

In [14]:
"""from toolz import take

with open('./data/Artworks.csv') as csvfile:
    reader = DictReader(csvfile, dialect=dialect)
    columns = reader.fieldnames
    head = list(take(2, reader))
head"""


"from toolz import take\n\nwith open('./data/Artworks.csv') as csvfile:\n    reader = DictReader(csvfile, dialect=dialect)\n    columns = reader.fieldnames\n    head = list(take(2, reader))\nhead"

## <font color="red"> Exercise 3 </font>

Create a file for the May Uber pick-ups that contains the various dateparts added in the last example.  Use the `bash` `head` function to inspect the first 10 rows of the result.  Include the ***day of the year*** and ***week of the year***.

In [15]:
# Your code here
c_size = 10000
new_names = ['date', 'lat', 'lon', 'base']
date_cols = ['date']
df_iter_m = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-may14.csv", 
                      header=0, names=new_names, 
                      parse_dates=date_cols,
                      chunksize=c_size,
                      sep=',',
                      engine='python')

In [17]:
from toolz import first
first_chunk_m = first(df_iter_m) 
first_chunk_m.dtypes

date    datetime64[ns]
lat            float64
lon            float64
base            object
dtype: object

In [18]:
(first_chunk_m 
 >> mutate(weekday = X.date.dt.weekday_name,
           weekofyear = X.date.dt.weekofyear,
           day = X.date.dt.day,
           hour = X.date.dt.hour) 
 >> head)

,date,lat,lon,base,weekday,weekofyear,day,hour
0,2014-05-01 00:02:00,40.7521,-73.9914,B02512,Thursday,18,1,0
1,2014-05-01 00:06:00,40.6965,-73.9715,B02512,Thursday,18,1,0
2,2014-05-01 00:15:00,40.7464,-73.9838,B02512,Thursday,18,1,0
3,2014-05-01 00:17:00,40.7463,-74.0011,B02512,Thursday,18,1,0
4,2014-05-01 00:17:00,40.7594,-73.9734,B02512,Thursday,18,1,0


In [19]:
from functoolz import pipeable
add_dateparts = pipeable(lambda df: (df 
                                     >> mutate(weekday = X.date.dt.weekday_name,
                                               weekofyear = X.date.dt.weekofyear,
                                               dayofyear = X.date.dt.dayofyear,
                                               year = X.date.dt.year,
                                               month = X.date.dt.month_name(),
                                               day = X.date.dt.day,
                                               hour = X.date.dt.hour)))
first_chunk_m >> add_dateparts >> head

,date,lat,lon,base,weekday,weekofyear,dayofyear,year,month,day,hour
0,2014-05-01 00:02:00,40.7521,-73.9914,B02512,Thursday,18,121,2014,May,1,0
1,2014-05-01 00:06:00,40.6965,-73.9715,B02512,Thursday,18,121,2014,May,1,0
2,2014-05-01 00:15:00,40.7464,-73.9838,B02512,Thursday,18,121,2014,May,1,0
3,2014-05-01 00:17:00,40.7463,-74.0011,B02512,Thursday,18,121,2014,May,1,0
4,2014-05-01 00:17:00,40.7594,-73.9734,B02512,Thursday,18,121,2014,May,1,0


In [20]:
out_file = "./data/uber-raw-data-may14-with-datepart.csv"
add_dateparts(first_chunk_m).to_csv(out_file, header=True, mode='w', index=False)

In [21]:
for i, chunk in enumerate(df_iter_m):
    print("writing chunk {0}".format(i+1))
    add_dateparts(chunk).to_csv(out_file, header=False, mode='a', index=False)

writing chunk 1
writing chunk 2
writing chunk 3
writing chunk 4
writing chunk 5
writing chunk 6
writing chunk 7
writing chunk 8
writing chunk 9
writing chunk 10
writing chunk 11
writing chunk 12
writing chunk 13
writing chunk 14
writing chunk 15
writing chunk 16
writing chunk 17
writing chunk 18
writing chunk 19
writing chunk 20
writing chunk 21
writing chunk 22
writing chunk 23
writing chunk 24
writing chunk 25
writing chunk 26
writing chunk 27
writing chunk 28
writing chunk 29
writing chunk 30
writing chunk 31
writing chunk 32
writing chunk 33
writing chunk 34
writing chunk 35
writing chunk 36
writing chunk 37
writing chunk 38
writing chunk 39
writing chunk 40
writing chunk 41
writing chunk 42
writing chunk 43
writing chunk 44
writing chunk 45
writing chunk 46
writing chunk 47
writing chunk 48
writing chunk 49
writing chunk 50
writing chunk 51
writing chunk 52
writing chunk 53
writing chunk 54
writing chunk 55
writing chunk 56
writing chunk 57
writing chunk 58
writing chunk 59
writin

In [22]:
!head -n 10 ./data/uber-raw-data-may14-with-datepart.csv

date,lat,lon,base,weekday,weekofyear,dayofyear,year,month,day,hour
2014-05-01 00:02:00,40.7521,-73.9914,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:06:00,40.6965,-73.9715,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:15:00,40.7464,-73.9838,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:17:00,40.7463,-74.0011,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:17:00,40.7594,-73.9734,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:20:00,40.7685,-73.8625,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:21:00,40.7637,-73.9962,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:21:00,40.7252,-74.0023,B02512,Thursday,18,121,2014,May,1,0
2014-05-01 00:25:00,40.7607,-73.9625,B02512,Thursday,18,121,2014,May,1,0


## Up Next

Stuff